# Загрузка Pandas и очистка данных

In [54]:
import pandas as pd

In [55]:
df = pd.read_csv('main_task.csv')

In [97]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 161 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Cuisine Style        40000 non-null  int64  
 1   Ranking              40000 non-null  float64
 2   Rating               40000 non-null  float64
 3   Price Range          40000 non-null  float64
 4   Number of Reviews    40000 non-null  float64
 5   Amsterdam            40000 non-null  uint8  
 6   Athens               40000 non-null  uint8  
 7   Barcelona            40000 non-null  uint8  
 8   Berlin               40000 non-null  uint8  
 9   Bratislava           40000 non-null  uint8  
 10  Brussels             40000 non-null  uint8  
 11  Budapest             40000 non-null  uint8  
 12  Copenhagen           40000 non-null  uint8  
 13  Dublin               40000 non-null  uint8  
 14  Edinburgh            40000 non-null  uint8  
 15  Geneva               40000 non-null

In [57]:
price_dict = {'$': 1, '$$ - $$$': 2.5, '$$$$': 4}
df['Price Range'] = df['Price Range'].replace(to_replace = price_dict)

In [58]:
df = pd.concat([df, pd.get_dummies(df['City'])], axis = 1)

In [62]:
cuisine_count = {}
for cuisines in df['Cuisine Style'].value_counts().index:
    for cuisine in cuisines.strip("[]").replace("'", "").split(', '):
        if cuisine in cuisine_count:
            cuisine_count[cuisine] += 1
        else:
            cuisine_count[cuisine] = 1

df['Cuisine Style'] = df['Cuisine Style'].fillna("['Vegetarian Friendly']")

df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: len(x.split(',')))

def find_item(cell):
    if type(cell) == str:
        if item in cell:
            return 1
    return 0

for item in cuisine_count.keys():
    df[item] = df['Cuisine Style'].apply(find_item)

In [64]:
df.drop(['City', 'Reviews', 'URL_TA', 'ID_TA'], axis = 1, inplace = True)

In [90]:
df['Number of Reviews'].fillna(df['Number of Reviews'].mode()[0], inplace = True)

In [93]:
df['Price Range'].fillna(df['Price Range'].mode()[0], inplace = True)

,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Amsterdam,Athens,Barcelona,Berlin,Bratislava,...,Southwestern,Uzbek,Canadian,Polynesian,Native American,Welsh,Fujian,Yunnan,Salvadoran,Xinjiang
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
39996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
39997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
39998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [99]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [100]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [101]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [102]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [103]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [104]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21347999999999998
